<a href="https://colab.research.google.com/github/yukaritk/heart_disease_project/blob/master/heart_disease_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>